In [222]:
import json
import sqlite3
import pandas as pd
import numpy as np

In [223]:
json_object = open('../data/spotify_kpop_train.json')
audiofeatures = json.load(json_object)

In [224]:
json_object = open('../data/lyric_sentiments.json')
lyric_sentiments = json.load(json_object)

In [225]:
json_object = open('../data/track_feeling_indicators.json')
track_feeling_indicators = json.load(json_object)

In [226]:
json_object = open('../data/words_of_interest_counter.json')
words_of_interest_counter = json.load(json_object)

In [227]:
json_object = open('../data/track_lyrics_translated.json')
track_lyrics_translated = json.load(json_object)

In [228]:
track_lyric_sentiments = []
for track in lyric_sentiments:
    track_id = list(track.keys())[0]
    
    lyric_line_sentiments = list(track.values())[0]
    
    sentiment_scores = []
    sentiments = []
    for lyric_sentiment in lyric_line_sentiments:
        sentiment = lyric_sentiment['sentiment'][0]
        sentiment_score = lyric_sentiment['sentiment'][1]
        sentiment_scores.append(sentiment_score)
        sentiments.append(sentiment)
    sentiment_mean_scores = np.mean(sentiment_scores)
    try: 
        positive_pct = sentiments.count('positive')/len(sentiments)
        neutral_pct = sentiments.count('neutral')/len(sentiments)
        negative_pct = sentiments.count('negative')/len(sentiments)
    except ZeroDivisionError:
        pass
    
    track_lyric_sentiments.append({'ID': track_id, 'lyric_sentiment_score': sentiment_mean_scores, 
                                   'lyric_positive': positive_pct, 'lyric_neutral': neutral_pct, 'lyric_negative': negative_pct})

/Users/ericyang/Desktop/spotify_kpop_ml/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ericyang/Desktop/spotify_kpop_ml/venv/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [229]:
lyric_sentiment_df = pd.DataFrame(track_lyric_sentiments)

In [230]:
track_feelings = []
for track in track_feeling_indicators:
    track_id = list(track.keys())[0]
    
    lyric_feelings = {}
    lyrics_indicators = list(track.values())[0]
    for lyric_indicators in lyrics_indicators:
        lyric_feelings_keys = list(lyric_feelings.keys())
        lyric_indicator_keys = list(lyric_indicators.keys())
        for lyric_indicator_key in lyric_indicator_keys:
            if lyric_indicator_key not in lyric_feelings_keys:
                lyric_feelings[lyric_indicator_key] = lyric_indicators[lyric_indicator_key]
            else:
                lyric_feelings[lyric_indicator_key] += lyric_indicators[lyric_indicator_key]
    lyric_feelings['ID'] = track_id
    track_feelings.append(lyric_feelings)

In [231]:
track_feelings_df = pd.DataFrame(track_feelings)

In [232]:
for feature in list(track_feelings_df.columns)[:-1]:
    track_feelings_df = track_feelings_df.rename(columns = {feature: f"lyric_{feature}"})

In [233]:
track_feelings_df.tail()

,lyric_question,lyric_exclamation,lyric_background_emphasis,lyric_emphasis,ID
295,5,0,0,0,3jsYQw78lrxJA2ysnmOIf9
296,0,0,0,0,15c7KZTrsCUxCQcOdUVELc
297,0,0,0,0,1Vd8qFWC07LB8UvNHyIlzg
298,0,0,6,35,4b9LMCUaw55QajVRfrfPyS
299,0,0,5,0,6NEdccy6N1RTqfjOHLkYk0


In [234]:
lyric_sentiment_df.head()

,ID,lyric_sentiment_score,lyric_positive,lyric_neutral,lyric_negative
0,3ObPkJQAgjAhTwYvDhPrAW,0.048145,0.238806,0.641791,0.119403
1,24nK8tW7Pt3Inh2utttuoG,0.034781,0.150943,0.698113,0.150943
2,1iIhGHzzrzqQfuNkFI2qAn,-0.068498,0.279070,0.255814,0.465116
3,4sa8O4AwMja6r1Enu35Fmz,0.041898,0.265306,0.571429,0.163265
4,4G5SXAQyz9D75KmKTLbI0X,0.114309,0.339286,0.607143,0.053571


In [235]:
words_of_interest = list(words_of_interest_counter.keys())

In [236]:
words_of_interest_clean = []
for word in words_of_interest:
    if "'" in word:
        word_clean = word.replace("'", "")
    else:
        word_clean = word
    words_of_interest_clean.append(word_clean)

In [237]:
lyric_word_features = {}
for word in words_of_interest_clean:
    lyric_word_features[word] = 0

In [238]:
track_ids = list(track_lyrics_translated.keys())
characters_to_remove = [',', '.', '?', '!', '-', '(', ')', '♪', "'"]


track_word_features = []
for track_id in track_ids:
    print(f"Track ID: {track_id}")
    lyric_lines = track_lyrics_translated[track_id]
    lyric_lines_lowercase = [lyric_line.lower() for lyric_line in lyric_lines]
    
    
    lyric_word_features = {}
    for word in words_of_interest_clean:
        lyric_word_features[word] = 0
        
    for lyric_line in lyric_lines_lowercase:
        
        for character_to_remove in characters_to_remove:
            lyric_line = lyric_line.replace(character_to_remove, '')
         
        words = lyric_line.split(' ')
        for word in words:
            if word in words_of_interest_clean:
                lyric_word_features[word] += 1
                    
    lyric_word_features['ID'] = track_id
    track_word_features.append(lyric_word_features)

Track ID: 3ObPkJQAgjAhTwYvDhPrAW
Track ID: 24nK8tW7Pt3Inh2utttuoG
Track ID: 1iIhGHzzrzqQfuNkFI2qAn
Track ID: 4sa8O4AwMja6r1Enu35Fmz
Track ID: 4G5SXAQyz9D75KmKTLbI0X
Track ID: 6puc8hVWY3mIE6x9SMEEzX
Track ID: 6txRAxfSDogqUd6BYzfocG
Track ID: 261LXApprrMeuczNAGqjav
Track ID: 0udhVlaFTeXvlmYWaFo66D
Track ID: 5Tl3pavJASY16gGDhCoe4Q
Track ID: 7Hjmz2XTMCuFvKBSfvrtC5
Track ID: 1bvOeyvMFzCNKIdvshyG2b
Track ID: 20bWnhRylVPnTpfsuIzorG
Track ID: 1AIYotQAJnVXpyfAznXK8y
Track ID: 0x9Vpw90j9fDb669IydnHL
Track ID: 7pbJi2eyN0cI0mbBkmLK90
Track ID: 2WusoL0p5HmFzhR5DLGUVx
Track ID: 6iZbX6zfmpx40UDy6Xcbl1
Track ID: 1ng3gM2zQ7kIJwC89np2oX
Track ID: 3hHpCjJdBBw4KjizY8l72r
Track ID: 0tWJ05IcctwDbJ0PfDbigg
Track ID: 3jsnWo73F2UuEtrZc6sHEx
Track ID: 2ljpZxz75NrBwunwnzRcId
Track ID: 1R8NDMQqxb2zLKUTlBoWuu
Track ID: 1MlGVEzfImYroIp15GShGj
Track ID: 1DF4MS2kTltHPabcFMM6ZE
Track ID: 7oIIKaSWTHkBpO5EVcV5Ac
Track ID: 4znsLjThGv9bBRQdEiMdTo
Track ID: 6GX6DixUAP2sJFfkBbVERG
Track ID: 1iArAuiiDPjtZcwO33YeLF
Track ID: 

In [239]:
track_word_features_df = pd.DataFrame(track_word_features)

In [240]:
track_word_features_df.head()

,i,baby,know,we,like,your,yeah,you,want,my,...,im,oh,love,now,dont,up,go,no,heart,ID
0,25,1,2,2,1,2,3,30,4,4,...,3,4,0,0,0,0,0,0,0,3ObPkJQAgjAhTwYvDhPrAW
1,14,0,0,0,1,2,4,8,0,4,...,0,3,1,1,5,9,2,0,0,24nK8tW7Pt3Inh2utttuoG
2,17,0,0,0,2,0,1,8,6,7,...,3,0,0,0,4,4,0,4,0,1iIhGHzzrzqQfuNkFI2qAn
3,10,1,4,4,0,0,4,13,3,7,...,3,1,0,0,1,5,0,0,0,4sa8O4AwMja6r1Enu35Fmz
4,14,1,1,0,1,4,1,15,3,7,...,1,3,1,1,0,0,2,0,1,4G5SXAQyz9D75KmKTLbI0X


In [241]:
for feature in list(track_word_features_df.columns)[:-1]:
    track_word_features_df = track_word_features_df.rename(columns = {feature: f"lyric_word_{feature}"})

In [242]:
track_word_features_df.head()

,lyric_word_i,lyric_word_baby,lyric_word_know,lyric_word_we,lyric_word_like,lyric_word_your,lyric_word_yeah,lyric_word_you,lyric_word_want,lyric_word_my,...,lyric_word_im,lyric_word_oh,lyric_word_love,lyric_word_now,lyric_word_dont,lyric_word_up,lyric_word_go,lyric_word_no,lyric_word_heart,ID
0,25,1,2,2,1,2,3,30,4,4,...,3,4,0,0,0,0,0,0,0,3ObPkJQAgjAhTwYvDhPrAW
1,14,0,0,0,1,2,4,8,0,4,...,0,3,1,1,5,9,2,0,0,24nK8tW7Pt3Inh2utttuoG
2,17,0,0,0,2,0,1,8,6,7,...,3,0,0,0,4,4,0,4,0,1iIhGHzzrzqQfuNkFI2qAn
3,10,1,4,4,0,0,4,13,3,7,...,3,1,0,0,1,5,0,0,0,4sa8O4AwMja6r1Enu35Fmz
4,14,1,1,0,1,4,1,15,3,7,...,1,3,1,1,0,0,2,0,1,4G5SXAQyz9D75KmKTLbI0X


In [244]:
audiofeatures_df = pd.DataFrame(audiofeatures)

In [247]:
audio_sentiment = pd.merge(audiofeatures_df, lyric_sentiment_df, on = 'ID')
audio_sentiment_feelings = pd.merge(audio_sentiment, track_feelings_df, on = 'ID')
final_df = pd.merge(audio_sentiment_feelings, track_word_features_df, on = 'ID')

In [251]:
conn = sqlite3.connect('../data/spotify_kpop.db')

In [252]:
final_df.to_sql('spotify_kpop_train', conn, index = False)

300